# Data obfuscation

This script removes personally identifiable information from the dataset.
This regards the following features:
- Submitter ID
- Project ID
- PR Number
- Closed At

Each of these variables are set to 0.

In addition, the dataframe's datapoint IDs are randomized, so no information can be derived from the order in which the data points are presented.

Because the `IsFirstTimeContributor` field is handled separately in `feature_construction`, it is merged into the dataset here to make sure the outputed obfuscated dataset is correct. Consequently, for this script to work properly, you need to have run the `feature_construction` script on the non-obfuscated data once already.

### Dataset loading

In [1]:
import pandas as pd
import python_proj.utils.exp_utils as exp_utils

input_data_file_name = "non_ftc_data"

data_path_format = exp_utils.BASE_PATH + "/final_data/{data_file_name}.csv"
data_path = data_path_format.format(data_file_name=input_data_file_name)
print(f"{data_path=}")


df: pd.DataFrame = pd.read_csv(filepath_or_buffer=data_path, header=0)

print(f"{len(df)=}.")
print(f"{len(df.columns)=}\n")

df.describe()

data_path='/workspaces/msc_thesis/data//final_data/non_ftc_data.csv'
len(df)=1815972.
len(df.columns)=155



,ID,Submitter ID,PR Number,ControlPullRequestLifeTimeInMinutes,ControlNumberOfCommitsInPullRequest,IntraProjectSecondOrderInDegreeCentrality(PRIntegratorToSubmitterV2.PRIntegratorToSubmitterV2-In),IntraProjectSecondOrderInDegreeCentrality(PRIntegratorToSubmitterV2.PRCommenterToSubmitterV2-In),IntraProjectSecondOrderInDegreeCentrality(PRIntegratorToSubmitterV2.PRCommenterToCommenterV2-In),IntraProjectSecondOrderInDegreeCentrality(PRIntegratorToSubmitterV2.IssueCommenterToCommenterV2-In),IntraProjectSecondOrderInDegreeCentrality(PRIntegratorToSubmitterV2.IssueCommenterToSubmitterV2-In),...,EcosystemSharedExperienceIssueSubmittedByIntegratorCommentedOnBySubmitter,EcosystemSharedExperienceIssueDiscussionParticipationByIntegratorAndSubmitter,EcosystemExperienceSubmitterIssueSubmissionCount,EcosystemExperienceSubmitterIssueCommentCount,DependencyEcosystemExperienceSubmitterIssueSubmissionCount,DependencyEcosystemExperienceSubmitterIssueCommentCount,NonDependencyEcosystemExperienceSubmitterIssueSubmissionCount,NonDependencyEcosystemExperienceSubmitterIssueCommentCount,InversedDependencyEcosystemExperienceSubmitterIssueSubmissionCount,InversedDependencyEcosystemExperienceSubmitterIssueCommentCount
count,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,...,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06
mean,1.474751e+08,5.198276e+06,3.807207e+03,3.190111e+04,4.947501e+00,3.926438e+01,1.076084e+01,1.262702e+01,8.083162e+00,8.484018e+00,...,8.594130e-02,1.475467e+00,2.615631e+00,1.785944e+01,1.743463e-01,8.645552e-01,2.274088e+00,1.570265e+01,1.854588e-01,1.355166e+00
std,1.004294e+08,8.100370e+06,8.071615e+03,1.399335e+05,7.283362e+01,2.497234e+03,3.186353e+02,4.506868e+02,3.652262e+02,1.962613e+02,...,1.285316e+00,2.382904e+01,7.985540e+00,6.700519e+01,1.580686e+00,1.186131e+01,7.064174e+00,6.131080e+01,2.446396e+00,1.667164e+01
min,6.870000e+02,1.000000e+00,1.000000e+00,1.666667e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.807322e+07,4.631570e+05,1.000000e+02,5.290000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.371940e+08,1.620265e+06,5.480000e+02,9.799167e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.278026e+08,6.384100e+06,2.817000e+03,7.328350e+03,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,2.000000e+00,7.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00
max,3.617756e+08,5.976123e+07,8.297600e+04,4.489490e+06,1.000000e+04,3.461280e+05,8.789200e+04,1.227920e+05,1.095040e+05,4.764000e+04,...,2.190000e+02,5.687000e+03,2.332000e+03,7.630000e+03,3.730000e+02,9.270000e+02,2.332000e+03,7.630000e+03,7.410000e+02,2.764000e+03


In [2]:
print(list(df.columns))

['ID', 'Project Name', 'Submitter ID', 'PR Number', 'Closed At', 'PullRequestIsMerged', 'SubmitterIsFirstTimeContributor', 'ControlIntegratedBySameUser', 'ControlPullRequestLifeTimeInMinutes', 'ControlPullRequestHasComments', 'ControlNumberOfCommitsInPullRequest', 'ControlPullRequestHasCommentByExternalUser', 'ControlHasHashTagInDescription', 'IntraProjectSecondOrderInDegreeCentrality(PRIntegratorToSubmitterV2.PRIntegratorToSubmitterV2-In)', 'IntraProjectSecondOrderInDegreeCentrality(PRIntegratorToSubmitterV2.PRCommenterToSubmitterV2-In)', 'IntraProjectSecondOrderInDegreeCentrality(PRIntegratorToSubmitterV2.PRCommenterToCommenterV2-In)', 'IntraProjectSecondOrderInDegreeCentrality(PRIntegratorToSubmitterV2.IssueCommenterToCommenterV2-In)', 'IntraProjectSecondOrderInDegreeCentrality(PRIntegratorToSubmitterV2.IssueCommenterToSubmitterV2-In)', 'IntraProjectSecondOrderInDegreeCentrality(PRCommenterToSubmitterV2.PRIntegratorToSubmitterV2-In)', 'IntraProjectSecondOrderInDegreeCentrality(PRCom

### Overwrite FTC data

In [3]:
def replace(
    df1: pd.DataFrame, df2: pd.DataFrame, match_column: str, overwritten_column: str
) -> pd.DataFrame:
    replace_data = df2[[match_column, overwritten_column]]

    old_counts = df1[overwritten_column].value_counts()

    df1 = df1.merge(
        replace_data, how="left", on=match_column, suffixes=("_incorrect", "")
    )
    old_key = f"{overwritten_column}_incorrect"
    df1 = df1.drop(old_key, axis=1)

    new_counts = df1[overwritten_column].value_counts()

    diff = new_counts - old_counts
    print(f"Impact overwriting '{overwritten_column}':\n{diff}")

    return df1


pre_existing_ftc_data_path = f"{exp_utils.BASE_PATH}/final_data/ftc_data.csv"
ftc_key = "SubmitterIsFirstTimeContributor"


def replace_with_pre_existing_data():
    global df
    print(f"{pre_existing_ftc_data_path=}\n")

    other_df: pd.DataFrame = pd.read_csv(pre_existing_ftc_data_path)
    df = replace(df, other_df, "ID", ftc_key)

    del other_df


print("Using FTC data file.")
replace_with_pre_existing_data()

Using FTC data file.
pre_existing_ftc_data_path='/workspaces/msc_thesis/data//final_data/ftc_data.csv'

Impact overwriting 'SubmitterIsFirstTimeContributor':
SubmitterIsFirstTimeContributor
False    215739
True    -215739
Name: count, dtype: int64


### Overwriting / removing problematic fields.

In [4]:
problematic_columns = ["Submitter ID", "PR Number", "Closed At"]

for col in problematic_columns:
    df[col] = 0

df[problematic_columns].describe()

,Project Name,Submitter ID,PR Number,Closed At
count,1815972.0,1815972.0,1815972.0,1815972.0
mean,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0


In [5]:
# Replaces project name with a number.

from wmutils.general import Counter

counter = Counter()

PROJ_NAME = "Project Name"

project_to_number = {name: counter.get_next() for name in df[PROJ_NAME].unique()}

df[PROJ_NAME] = df[PROJ_NAME].replace(project_to_number)

In [6]:
# Resets the data point ID.

df = df.sample(frac=1).reset_index(drop=True)
df = df.drop(["ID"], axis=1)

In [7]:
import python_proj.utils.exp_utils as exp_utils

output_file = f"{exp_utils.BASE_PATH}/final_data/non_ftc_data_obfuscated.csv"
df.to_csv(output_file, index_label='ID')

KeyboardInterrupt: 

In [ ]:
import datetime

print(datetime.datetime.now())

2024-02-21 15:52:09.375174
